In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        #input 1 input channel, 6 output channel, 3 x 3 kernel
        self.conv1 = nn.Conv2d(1,6,3)
        #input 6 input channel, 16 output channel, 3 x 3 kernel
        self.conv2 =  nn.Conv2d(6,16,3)

        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self,x):
        #nn inception
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features*= s

        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [5]:
input = torch.rand(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0422, -0.0935, -0.0177, -0.0656,  0.1272,  0.0269, -0.0420, -0.0647,
          0.0211,  0.0918]], grad_fn=<AddmmBackward>)


In [6]:
net.zero_grad()
out.backward(torch.rand(1,10))

In [7]:
output = net(input)
target = torch.rand(1,10)
target = target.view(1,-1)
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor(0.2611, grad_fn=<MseLossBackward>)


In [8]:
loss.grad_fn.next_functions[0][0]
loss.grad_fn.next_functions[0][0] #relu



In [9]:
net.zero_grad()

print('conv1.bais.grad before the backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after the backward')
print(net.conv1.bias.grad)


conv1.bais.grad before the backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after the backward
tensor([ 0.0025,  0.0060, -0.0056,  0.0011, -0.0040,  0.0038])


In [10]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update